In [1]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial


import numpy as np

def module(num):
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean((x - y))
    
def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))


def recursive_act(act_list, indent):
    current_trial = __noworkflow__.trial_id
    
    if act_list:
        act = act_list.pop(0)
        if act.type in ('func', 'assign'):           
            if act.type == 'func':
                sub_act = Activation((current_trial, act.dependent_id))
                print(' ' * indent + "actv id = "+ str(sub_act.id) + '; function = ' + str(sub_act.name) + '; code_block = ' + str(sub_act.code_block_id))
                
            if act.type == 'assign':
                sub_eval = Evaluation((current_trial, act.dependent_id))
                code_comp = sub_eval.code_component
                if (code_comp.type == 'call') or ((code_comp.mode == 'w') and (code_comp.type != 'param')): 
                    op_name = code_comp.name
                    print(' ' * indent + "eval id = "+ str(code_comp.id) + "; variable = " + code_comp.name + ' <- ' + str(sub_eval.repr))
        recursive_act(act_list, indent)
    else:
        print("- - - - -")

In [2]:
# What I want: get the id of a given cell, capture assingnment and function calls inside this cell and print
# the output

x = 10
y = -100
z = 50

rmse_err = rmse(x, y)
mae_err = mae(x, z)
modl_err = module(-3)

last_activation = __noworkflow__.last_activation

In [3]:
rmse_err, mae_err, modl_err

(110.0, -40.0, 3)

In [4]:
last_activation

Activation(id=10, name=<ipython-input-2-321b278837f9>, start_checkpoint=5.718529616, code_block_id=435)

In [5]:
activation = Activation((__noworkflow__.trial_id, 10))
act_list = list(activation.dependent_dependencies)

recursive_act(act_list, 1)

 eval id = 438; variable = x <- 10
 eval id = 442; variable = y <- -100
 eval id = 448; variable = z <- 50
 actv id = 22; function = rmse; code_block = 77
 eval id = 452; variable = rmse_err <- 110.0
 actv id = 43; function = mae; code_block = 55
 eval id = 462; variable = mae_err <- -40.0
 actv id = 56; function = module; code_block = 20
 eval id = 472; variable = modl_err <- 3
 eval id = 482; variable = last_activation <- Activation(id=10, name=<ipython-input-2-321b278837f9>, start_checkpoint=5.718529616, code_block_id=435)
- - - - -


Observations:

1. The function is recursive in only one direction. Trying to search backwards will break it
2. It only displays the history of what was executed inside the cell. It doesn't attends the need of printing the dependencies of
    a given variable. 

The usecase here is:
When tagging one variable, I want to be able to capture all the dependencies of this variable accross the trial.
1. I have to iterate over all the executed cells in the notebook
2. Capture all references to this variabble
3. Capture all references to the references of this variable
4. DIsplay it at the end.